In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

df_eg= pd.read_csv('./datasets/egypt _store.csv')
df_fr= pd.read_csv('./datasets/french_store.csv')
df_sa= pd.read_csv('./datasets/sa_store.csv')


Clean Egypt Store (Jumia)

In [2]:
df_eg.head()

,selection1_url,selection1_name,selection1_discount,selection1_price,selection1_brand,selection1_brand_url
0,https://www.jumia.com.eg/g3-15-3500-gaming-lap...,DELL G3 15-3500 Gaming Laptop - Intel Core I5 ...,18%,"EGP 15,495",DELL,https://www.jumia.com.eg/dell/
1,https://www.jumia.com.eg/nitro-5-laptop-intel-...,Acer Nitro 5 Laptop - Intel Core I5 10TH - 8GB...,19%,"EGP 14,499",Acer,https://www.jumia.com.eg/acer/
2,https://www.jumia.com.eg/legion-5-15imh05h-gam...,Lenovo Legion 5-15IMH05H Gaming Laptop - Intel...,20%,"EGP 19,880",Lenovo,https://www.jumia.com.eg/lenovo/
3,https://www.jumia.com.eg/dell-g5-15-5500-gamin...,DELL G5 15-5500 Gaming Laptop - Intel Core I7 ...,16%,"EGP 16,898",DELL,https://www.jumia.com.eg/dell/
4,https://www.jumia.com.eg/legion-5-15imh05-gami...,Lenovo Legion 5-15IMH05 Gaming Laptop - Intel ...,11%,"EGP 16,849",Lenovo,https://www.jumia.com.eg/lenovo/


In [3]:
# rename columns
df_eg.columns = ['url', 'text', 'discount', 'price', 'brand', 'brand_url']
df_eg.isna().sum()

url          0
text         0
discount     2
price        0
brand        0
brand_url    0
dtype: int64

In [4]:
# remove rows with nan values 
df_eg = df_eg[df_eg['discount'].isna() == False]
assert df_eg.isna().sum().sum() == 0, 'there are still nan values in df_eg'

In [5]:
# turn price into integers
df_eg.price.describe()
def extract_price(string):
    reg_expression= r"(?:\d+\,\d+|\d+)"
    title_search = re.search(reg_expression, string, re.IGNORECASE)
    #print(title_search[0])
    res= title_search[0].replace(',', '')
    res= int(res)
    return res
extract_price('EGP 123,345 EGB 0')
price_value=df_eg.price.map(lambda x: extract_price(x))
df_eg.rename(columns = {'price':'original_price_EPG'}, inplace = True)
df_eg['price']= price_value
df_eg.head(1)

,url,text,discount,original_price_EPG,brand,brand_url,price
0,https://www.jumia.com.eg/g3-15-3500-gaming-lap...,DELL G3 15-3500 Gaming Laptop - Intel Core I5 ...,18%,"EGP 15,495",DELL,https://www.jumia.com.eg/dell/,15495


In [6]:
# get value of discount
def extract_discount_value(string):
    reg_expression= r"(?:\d+\,\d+|\d+)"
    title_search = re.search(reg_expression, string, re.IGNORECASE)
    #print(title_search[0])
    res= title_search[0]
    res= int(res)
    return res
df_eg['discount_value']=df_eg.discount.map(lambda x: extract_discount_value(x))
df_eg.head(1)

,url,text,discount,original_price_EPG,brand,brand_url,price,discount_value
0,https://www.jumia.com.eg/g3-15-3500-gaming-lap...,DELL G3 15-3500 Gaming Laptop - Intel Core I5 ...,18%,"EGP 15,495",DELL,https://www.jumia.com.eg/dell/,15495,18


In [7]:
# extract ram 
def extract_ram(string):
    reg_expression_ram= '\d+ *GB RAM'
    ram= re.search(reg_expression_ram, string, re.IGNORECASE)
    reg_exp_extract_number= '\d+'
    ram_value= re.search(reg_exp_extract_number, ram[0], re.IGNORECASE) if ram else [np.nan]
    return ram_value[0]
df_eg['ram_value']= df_eg.text.map(extract_ram)
# remove rows where the ram was not present
df_eg= df_eg[~df_eg.ram_value.isna()]

In [8]:
# extract wheather it is an intel or AMD CPU and extract cpu name and generation
def extract_cpu_type(string):
    """matches I7, I5, I3 for intel and Ryzen 5, 6, 7 for AMD and returns (name_of_the_cpu, name_of_manufacturer)"""
    reg_expression_intel= r"I(\d)"
    intel_match = re.search(reg_expression_intel, string, re.IGNORECASE)
    reg_expression_amd= r"Ryzen (\d)"
    AMD_match = re.search(reg_expression_amd, string, re.IGNORECASE)
    if (intel_match and AMD_match):
        raise Exception("Sorry, cant match properly")
    if(intel_match):
        cpu_name= intel_match[0]
        cpu_number= intel_match[1]
        cpu_brand= 'Intel'
        return cpu_name, cpu_number, cpu_brand
    if(AMD_match):
        cpu_name= AMD_match[0]
        cpu_number= AMD_match[1]
        cpu_brand= 'AMD'
        return cpu_name, cpu_number, cpu_brand
    return np.nan, np.nan, np.nan
extract_cpu_type('Lenovo Legion 5 15IMH05H Gaming Laptop - Intel Core I7 - 16GB RAM - 256GB SSD + 1TB HDD - 15.6-inch FHD - 6GB GTX1660 Ti GPU - DOS - Phantom Black')
extract_cpu_type('Lenovo IdeaPad Gaming 3 15ARH05 Laptop - AMD Ryzen 5 - 16GB RAM - 256GB SSD + 1TB HDD - 15.6-inch FHD - 4GB GPU - DOS - Chameleon Blue')
extract_cpu_type('Lenovo IdeaPad Gaming 3 15ARH05 Laptop ameleon Blue')

df_eg['cpu_name'] =df_eg.text.map(lambda x: extract_cpu_type(x)[0])
df_eg['cpu_gen'] =df_eg.text.map(lambda x: extract_cpu_type(x)[1])
df_eg['cpu_brand'] =df_eg.text.map(lambda x: extract_cpu_type(x)[2])


In [9]:
df_eg.sample(5).text.values

array(['Lenovo Legion Y540-15IRH Gaming Laptop - Intel Core I7 - 16GB RAM - 1TB HDD + 256GB SSD - 15.6-inch FHD - 6GB GPU - DOS - Black',
       'Asus ROG ZEPHYRUS M15 GU502LW Gaming Laptop - Intel Core I7 - 16GB RAM - 512GB SSD - 15.6-inch FHD - 8GB GPU - Windows 10 - Black (English Keyboard)',
       'Lenovo Ideapad 3 Gaming Laptop - Intel Core I7 - 8GB RAM - 512 SSD - 15.6-inch FHD - 4GB GPU - Windows 10 - Black',
       'Lenovo Legion 5-15IMH05 Gaming Laptop - Intel Core I7 - 16GB RAM - 1TB HDD + 512GB SSD - 15.6-inch FHD - 4GB GPU - Windows 10 - Phantom Black (English Keyboard)',
       'DELL G5 15-5500 Gaming Laptop - Intel Core I7-10750H - 16GB RAM - 512GB SSD - 15.6-inch FHD 120Hz - GTX 1660Ti 6GB GDDR6 -Ubuntu - Black'],
      dtype=object)

Clean saudi arabia site

1. remove extra url column as they both point to the same location

In [10]:
df_sa.columns = ['text', 'url', 'price', 'url2']
df_sa.head(2)

,text,url,price,url2
0,لاب توب العاب من ام اس اي جي اس 66 ستيلث 10SFS...,https://www.amazon.sa/%D8%A7%D9%84%D8%B9%D8%A7...,"11,377.16 ريالريال‎11,377.‎16‏",https://www.amazon.sa/%D8%A7%D9%84%D8%B9%D8%A7...
1,لاب توب ام اس اي للالعاب GL65 ليوبارد 10SFK-06...,https://www.amazon.sa/%D9%84%D9%84%D8%A7%D9%84...,"5,918.53 ريالريال‎5,918.‎53‏",https://www.amazon.sa/%D9%84%D9%84%D8%A7%D9%84...


In [11]:
# make sure url2 is nothing more than url
print(f'there are {df_sa[df_sa.url != df_sa.url2].shape} rows where url not equal url2 because they are NAN')
# drop url2
df_sa.drop('url2', axis=1,inplace=True)

there are (18, 4) rows where url not equal url2 because they are NAN


In [12]:
# extract cpu name, gen, brand
# extract wheather it is an intel or AMD CPU and extract cpu name and generation
def extract_cpu_type_arabic_and_english(string):
    """extracts intel cpu name and gen even if it is writen in arabic, english, in franco (arabic letters with english pronounciation)"""
    reg_expression_intel= r"(?: اي(\d)| اي (\d)|i(\d))"
    intel_match = re.search(reg_expression_intel, string, re.IGNORECASE)
    reg_expression_amd= r"(?:رايزن(\d)|رايزن (\d)|Ryzen(\d)|Ryzen (\d))"
    AMD_match = re.search(reg_expression_amd, string, re.IGNORECASE)
    if (intel_match and AMD_match):
        raise Exception("matched both ryzen and intel")
    if(intel_match):
        cpu_name= intel_match[0]
        cpu_number= intel_match[1] or intel_match[2] or intel_match[3] #extract from any pattern the 1st non None match
        cpu_brand= 'Intel'
        return cpu_name, cpu_number, cpu_brand
    if(AMD_match):
        cpu_name= AMD_match[0]
        cpu_number= AMD_match[1] or AMD_match[2] or AMD_match[3] or AMD_match[4] #extract from any pattern the 1st non None match
        cpu_brand= 'AMD'
        return cpu_name, cpu_number, cpu_brand
    return np.nan, np.nan, np.nan

df_sa['cpu_name'] =df_sa.text.map(lambda x: extract_cpu_type_arabic_and_english(x)[0])
df_sa['cpu_gen'] =df_sa.text.map(lambda x: extract_cpu_type_arabic_and_english(x)[1])
df_sa['cpu_brand'] =df_sa.text.map(lambda x: extract_cpu_type_arabic_and_english(x)[2])


In [13]:
df_sa.head()

,text,url,price,cpu_name,cpu_gen,cpu_brand
0,لاب توب العاب من ام اس اي جي اس 66 ستيلث 10SFS...,https://www.amazon.sa/%D8%A7%D9%84%D8%B9%D8%A7...,"11,377.16 ريالريال‎11,377.‎16‏",اي 9,9,Intel
1,لاب توب ام اس اي للالعاب GL65 ليوبارد 10SFK-06...,https://www.amazon.sa/%D9%84%D9%84%D8%A7%D9%84...,"5,918.53 ريالريال‎5,918.‎53‏",i7,7,Intel
2,سماعة راس للالعاب HS35 من كورسير - سماعات راس ...,https://www.amazon.sa/%D8%B3%D9%85%D8%A7%D8%B9...,229.00 ريالريال‎229.‎00‏,NaN,NaN,NaN
3,لاب توب لينوفو ليجون 5 15.6 انش فل اتش دي (192...,https://www.amazon.sa/%D9%84%D9%8A%D9%86%D9%88...,"4,129.11 ريالريال‎4,129.‎11‏",رايزن 7,7,AMD
4,لاب توب ايسر نيترو 5 الجيل التاسع انتل كورi5-9...,https://www.amazon.sa/%D8%A7%D9%8A%D8%B3%D8%B1...,"3,240.10 ريالريال‎3,240.‎10‏",i5,5,Intel


In [14]:
no_cpu_name= df_sa[df_sa.cpu_name.isna()]
no_cpu_gen= df_sa[df_sa.cpu_gen.isna()]
no_cpu_brand= df_sa[df_sa.cpu_brand.isna()]
print(f'number of rows without cpu_name= {no_cpu_name.shape[0]}')
print(f'number of rows without cpu_gen= {no_cpu_gen.shape[0]}')
print(f'number of rows without cpu_brand= {no_cpu_brand.shape[0]}')
print('sanity check and to make sure these should be removed as they lack a CPU either because of incomplete info or because they are laptop accessories like this one')
print(no_cpu_brand.head(1).text.values)
print('this translates to a headset not a laptop')

number of rows without cpu_name= 214
number of rows without cpu_gen= 214
number of rows without cpu_brand= 214
sanity check and to make sure these should be removed as they lack a CPU either because of incomplete info or because they are laptop accessories like this one
['سماعة راس للالعاب HS35 من كورسير - سماعات راس بصوت مضخم للالعاب - وسادات اذن من رغوة الذاكرة - سماعات الراس مصممة لجهاز Xbox one والموبايل - اخضر']
this translates to a headset not a laptop


In [15]:
#drop extra rows with incomplete data
df_sa= df_sa[~df_sa.cpu_name.isna()]
print(f'size of dataset after cleaning extra rows {df_sa.shape[0]}')

size of dataset after cleaning extra rows 74


In [16]:
# extract price

price_value= df_sa.price.map(lambda x: extract_price(x))
df_sa.rename({'price': 'original_price_SAR'}, inplace=True)
df_sa['price']= price_value
df_sa.head()

,text,url,price,cpu_name,cpu_gen,cpu_brand
0,لاب توب العاب من ام اس اي جي اس 66 ستيلث 10SFS...,https://www.amazon.sa/%D8%A7%D9%84%D8%B9%D8%A7...,11377,اي 9,9,Intel
1,لاب توب ام اس اي للالعاب GL65 ليوبارد 10SFK-06...,https://www.amazon.sa/%D9%84%D9%84%D8%A7%D9%84...,5918,i7,7,Intel
3,لاب توب لينوفو ليجون 5 15.6 انش فل اتش دي (192...,https://www.amazon.sa/%D9%84%D9%8A%D9%86%D9%88...,4129,رايزن 7,7,AMD
4,لاب توب ايسر نيترو 5 الجيل التاسع انتل كورi5-9...,https://www.amazon.sa/%D8%A7%D9%8A%D8%B3%D8%B1...,3240,i5,5,Intel
5,لاب توب ايسر بريداتور هيلوس 300 للالعاب انتل i...,https://www.amazon.sa/%D8%A7%D9%8A%D8%B3%D8%B1...,7550,i7,7,Intel


In [17]:
# extract ram 
def extract_ram(string):
    reg_expression_ram= '\d+ *GB *RAM'
    ram= re.search(reg_expression_ram, string, re.IGNORECASE)
    reg_exp_extract_number= '\d+'
    ram_value= re.search(reg_exp_extract_number, ram[0], re.IGNORECASE) if ram else [np.nan]
    return ram_value[0]
df_sa['ram_value']= df_sa.text.map(extract_ram)
df_sa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 261
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       74 non-null     object
 1   url        74 non-null     object
 2   price      74 non-null     int64 
 3   cpu_name   74 non-null     object
 4   cpu_gen    74 non-null     object
 5   cpu_brand  74 non-null     object
 6   ram_value  24 non-null     object
dtypes: int64(1), object(6)
memory usage: 4.6+ KB


Clean french store (boulanger)

In [18]:
df_fr.head(3)
df_fr.columns= ['text', 'url', 'display', 'display2', 'refresh_rate', 'cpu', 'storage', 'ram', 'price', 'earliest_arrival', 'earliest_arrival2', 'ports', 'price_before_discount', 'discount', 'stock']

In [19]:
df_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   text                   100 non-null    object
 1   url                    100 non-null    object
 2   display                100 non-null    object
 3   display2               100 non-null    object
 4   refresh_rate           100 non-null    object
 5   cpu                    100 non-null    object
 6   storage                100 non-null    object
 7   ram                    100 non-null    object
 8   price                  100 non-null    object
 9   earliest_arrival       100 non-null    object
 10  earliest_arrival2      68 non-null     object
 11  ports                  91 non-null     object
 12  price_before_discount  100 non-null    object
 13  discount               100 non-null    object
 14  stock                  23 non-null     object
dtypes: object(15)
memory usa

In [20]:
# drop stock column as it is mostly empty
df_fr.drop('stock', axis=1, inplace=True)
# drop earliest_arrival2 as it is a duplicate column
df_fr.drop('earliest_arrival2', axis=1, inplace=True)
# drop display 2
df_fr.drop('display2', axis=1, inplace=True )

In [21]:
df_fr.head(1)

,text,url,display,refresh_rate,cpu,storage,ram,price,earliest_arrival,ports,price_before_discount,discount
0,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tDell\n\...,https://www.boulanger.com/ref/1152079,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go,"Mémoire vive 8 Go - 2,3 kg","899,99€",chez vous dès vendredi 12 mars,Le + : USB C + Clavier rétro-éclairé,"1199,00€",- 24%


In [22]:
# extract cpu info
df_fr['cpu_name']= df_fr.cpu.map(lambda x: extract_cpu_type(x)[0])
df_fr['cpu_gen']= df_fr.cpu.map(lambda x: extract_cpu_type(x)[1])
df_fr['cpu_brand']= df_fr.cpu.map(lambda x: extract_cpu_type(x)[2])
df_fr[['cpu_name', 'cpu_gen', 'cpu_brand']].isna().sum()
sus_rows= df_fr[df_fr.cpu_name.isna() | df_fr.cpu_gen.isna()| df_fr.cpu_brand.isna()]
print(f'there are {sus_rows.shape[0]} rows where the cpu info was not extracted')

there are 18 rows where the cpu info was not extracted


Further inspection shows that for these rows while the data was gatherd the cpu section was inputed in the refresh_rate section
so we will extract the cpu info frmo the right column and then add them back to the dataset

In [23]:
%%capture --no-display 
sus_rows.sample()[['refresh_rate', 'cpu']]
sus_rows['cpu_name']= sus_rows.refresh_rate.map(lambda x: extract_cpu_type(x)[0])
sus_rows['cpu_gen']= sus_rows.refresh_rate.map(lambda x: extract_cpu_type(x)[1])
sus_rows['cpu_brand']= sus_rows.refresh_rate.map(lambda x: extract_cpu_type(x)[2])
df_fr= df_fr[~(df_fr.cpu_name.isna() | df_fr.cpu_gen.isna()| df_fr.cpu_brand.isna())] # drop those with nan values
df_fr.append(sus_rows)
# df_fr[df_fr.cpu_name.isna() | df_fr.cpu_gen.isna()| df_fr.cpu_brand.isna()] = sus_rows

,text,url,display,refresh_rate,cpu,storage,ram,price,earliest_arrival,ports,price_before_discount,discount,cpu_name,cpu_gen,cpu_brand
0,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tDell\n\...,https://www.boulanger.com/ref/1152079,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go,"Mémoire vive 8 Go - 2,3 kg","899,99€",chez vous dès vendredi 12 mars,Le + : USB C + Clavier rétro-éclairé,"1199,00€",- 24%,i5,5,Intel
1,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1151227,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,Disque dur 1 To et SSD 512 Go,"Mémoire vive 16 Go - 2,9 kg","1699,00€",chez vous dès vendredi 12 mars,"Le + : Processeur 10 ème génération, Système ...","1999,00€",- 15%,i7,7,Intel
2,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157099,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,SSD 1 To,"Mémoire vive 32 Go - 2,3 kg","1899,00€",chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3060 GDDR6 6 Go,"1899,00€",0%,i7,7,Intel
3,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tDell\n\...,https://www.boulanger.com/ref/1152078,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go,"Mémoire vive 8 Go - 2,3 kg","799,99€",chez vous dès vendredi 12 mars,Le + : Game Shift déclenche un mode de perfor...,"899,00€",- 11%,i5,5,Intel
4,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1157998,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go (en PCi-Express),"Mémoire vive 8 Go - 2,1 kg","1099,00€",chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce GTX1660Ti DDR6 6Go,"1299,00€",- 15%,i5,5,Intel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tHP\n\t\...,https://www.boulanger.com/ref/1071802,"15,6"" Full HD (39,6 cm)","Intel Core i7-6700HQ : 2,6 GHz ; Turbo 3,5 GHz...",Disque dur 2 To et SSD 128 Go,"2,2 kg","2,2 kg","983,39€",sous 6 jours ouvrés,Le + : NVIDIA GeForce GTX960M avec 4 Go GDDR5,"983,39€",0%,i7,7,Intel
81,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tHP\n\t\...,https://www.boulanger.com/ref/1079055,"17,3"" Full HD (43,9 cm)","Intel Core i7-6700HQ : 2,6 GHz ; Turbo 3,5 GHz...",Disque dur 1 To et SSD 128 Go,"3,4 kg","3,4 kg","1009,79€",sous 6 jours ouvrés,Le + : NVIDIA GeForce GTX1070 avec 8 Go GDDR5,"1009,79€",0%,i7,7,Intel
84,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tLenovo\...,https://www.boulanger.com/ref/1123506,"15,6"" Full HD (39,6 cm)","Intel Core i7 8750H : 2,2 GHz ; Turbo 4,1 GHz ...",Disque dur 1 To et SSD 256 Go,16 Go,"Mémoire vive 16 Go - 2,2 kg","1073,49€",sous 6 jours ouvrés,Le + : NVIDIA GeForce RTX 2060 6Go,"1073,49€",0%,i7,7,Intel
86,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1091327,"17,3"" Full HD (43,9 cm)","Intel Core i5-7300HQ : 2,5 GHz ; Turbo 3,5 GHz...",Disque dur 1 To et SSD 256 Go,"3,0 kg","3,0 kg","1075,49€",sous 6 jours ouvrés,Le + : NVIDIA GeForce GTX1050 TI avec 4 Go GDDR5,"1075,49€",2%,i5,5,Intel


In [24]:
print(f'number of nan values for extracted cpu info is {df_fr[df_fr.cpu_name.isna() | df_fr.cpu_gen.isna()| df_fr.cpu_brand.isna()].shape[0]}')

number of nan values for extracted cpu info is 0


In [25]:

# extract prices
def extract_price_fr(string):
    """extracts price but as it will be used for a frnech site it follows the french convention of comma being the decimal seperator"""
    reg_expression= r"(?:\d+\,\d+|\d+)"
    title_search = re.search(reg_expression, string, re.IGNORECASE)
    #print(title_search[0])
    res= title_search[0].replace(',', '.')
    res= int(round( float(res) ) )
    return res
price_value= df_fr.price.map(lambda x: extract_price_fr(x))
df_fr.rename({'price':'original_price_EUR'}, inplace=True)
df_fr['price']= price_value
df_fr.head()


,text,url,display,refresh_rate,cpu,storage,ram,price,earliest_arrival,ports,price_before_discount,discount,cpu_name,cpu_gen,cpu_brand
0,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tDell\n\...,https://www.boulanger.com/ref/1152079,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go,"Mémoire vive 8 Go - 2,3 kg",900,chez vous dès vendredi 12 mars,Le + : USB C + Clavier rétro-éclairé,"1199,00€",- 24%,i5,5,Intel
1,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1151227,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,Disque dur 1 To et SSD 512 Go,"Mémoire vive 16 Go - 2,9 kg",1699,chez vous dès vendredi 12 mars,"Le + : Processeur 10 ème génération, Système ...","1999,00€",- 15%,i7,7,Intel
2,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157099,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,SSD 1 To,"Mémoire vive 32 Go - 2,3 kg",1899,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3060 GDDR6 6 Go,"1899,00€",0%,i7,7,Intel
3,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tDell\n\...,https://www.boulanger.com/ref/1152078,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go,"Mémoire vive 8 Go - 2,3 kg",800,chez vous dès vendredi 12 mars,Le + : Game Shift déclenche un mode de perfor...,"899,00€",- 11%,i5,5,Intel
4,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1157998,"15,6"" Full HD (39,6 cm) - 120 Hz",120 Hz,"Intel Core i5 - 10300H : 2,5 GHz / Turboboost ...",SSD 512 Go (en PCi-Express),"Mémoire vive 8 Go - 2,1 kg",1099,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce GTX1660Ti DDR6 6Go,"1299,00€",- 15%,i5,5,Intel


In [26]:
# extract discounted value
df_fr['discount_value']= df_fr.discount.map(lambda x: extract_discount_value(x)) 

In [27]:
# clean the ram column and extract the weight column from it 
df_fr.ram.value_counts()

Mémoire vive 16 Go - 2,4 kg    7
Mémoire vive 16 Go - 2,3 kg    7
Mémoire vive 16 Go - 2,6 kg    6
Mémoire vive 16 Go - 1,7 kg    5
Mémoire vive 16 Go - 2,2 kg    4
Mémoire vive 32 Go - 2,5 kg    4
Mémoire vive 8 Go - 2,3 kg     4
Mémoire vive 32 Go - 2,4 kg    4
Mémoire vive 16 Go - 2,9 kg    3
Mémoire vive 32 Go - 2,1 kg    3
Mémoire vive 32 Go - 2,9 kg    3
Mémoire vive 32 Go - 2,3 kg    3
Mémoire vive 16 Go - 4,7 kg    2
Mémoire vive 32 Go - 3,0 kg    2
Mémoire vive 16 Go - 2,0 kg    2
Mémoire vive 32 Go - 2,6 kg    2
Mémoire vive 8 Go - 3,0 kg     1
Mémoire vive 16 Go - 1,6 kg    1
Mémoire vive 16 Go - 2,1 kg    1
2,6 kg                         1
Mémoire vive 8 Go - 2,5 kg     1
Mémoire vive 16 Go - 1,3 kg    1
Mémoire vive 8 Go - 2,6 kg     1
Mémoire vive 16 Go - 2,8 kg    1
Mémoire vive 16 Go - 2,7 kg    1
Mémoire vive 64 Go - 2,9 kg    1
2,2 kg                         1
Mémoire vive 8 Go - 3,8 kg     1
Mémoire vive 8 Go - 2,1 kg     1
Mémoire vive 32 Go - 3,2 kg    1
Mémoire vi

In [28]:
def extract_weight(string):
    string= string.replace(',','.')
    reg_expression= '\d+.\d* *kg'
    weight= re.search(reg_expression, string, re.IGNORECASE)[0]
    weight_value= float(weight.replace('kg', '').strip())
    return weight_value
def extract_ram(string):
    reg_expression_ram= 'Mémoire vive \d+ Go'
    ram= re.search(reg_expression_ram, string, re.IGNORECASE)
    reg_exp_extract_number= '\d+'
    ram_value= re.search(reg_exp_extract_number, ram[0], re.IGNORECASE) if ram else [np.nan]
    return ram_value[0]
df_fr.ram.map(lambda x:extract_weight(x)).value_counts()
df_fr.ram.map(lambda x:extract_ram(x)).value_counts()
df_fr['ram_value']= df_fr.ram.map(lambda x:extract_ram(x))
df_fr['weight_KG']= df_fr.ram.map(lambda x:extract_weight(x))
df_fr.ram_value.value_counts()

16    45
32    23
8     11
64     1
Name: ram_value, dtype: int64

Merge all the Datasets into 1 unified Dataset
1. on the Sa site there were no discounts so we will assume discounts= 0%
2. make sure to unif price in dollars as each store uses the home contries curency

    _Exchange Rates (7/28/2021)_
    - 1 EPG = 0.06 USD
    - 1 EUR = 1.18 USD
    - 1 SAR = 0.27 USD

In [29]:
df_sa['discount_value'] = 0
df_sa['store']= 'Amazon (SA)'
exchange_factor_sa= 0.27
df_sa.price= df_sa.price.map(lambda x: round(x*exchange_factor_sa))
df_sa.head(1)

,text,url,price,cpu_name,cpu_gen,cpu_brand,ram_value,discount_value,store
0,لاب توب العاب من ام اس اي جي اس 66 ستيلث 10SFS...,https://www.amazon.sa/%D8%A7%D9%84%D8%B9%D8%A7...,3072,اي 9,9,Intel,NaN,0,Amazon (SA)


In [30]:
df_fr.price.describe()

count      82.000000
mean     2020.256098
std       883.883474
min       800.000000
25%      1384.000000
50%      1898.500000
75%      2374.000000
max      4999.000000
Name: price, dtype: float64

In [31]:
df_fr['store']= 'Boulanger (FR)'
exchange_factor_fr= 1.18
#df_fr.price= df_fr.price.map(lambda x: round(x*exchange_factor_fr))
df_fr.sample(5)
#df_fr.price.describe()
print(round(1699*exchange_factor_fr))
df_fr['price']= df_fr.price.map(lambda x: round(x*exchange_factor_fr))
df_fr.sample(5)

2005


,text,url,display,refresh_rate,cpu,storage,ram,price,earliest_arrival,ports,price_before_discount,discount,cpu_name,cpu_gen,cpu_brand,discount_value,ram_value,weight_KG,store
54,Ordinateur portable \n\t\t\t\t\t\t\n\t\t\t\t\t...,https://www.boulanger.com/ref/1157078,"15,6"" 4K (39,6 cm) - 60 Hz",60 Hz,"Intel Core i7 - 10875H : 2,3 GHz / Turboboost ...",SSD 2 To,"Mémoire vive 32 Go - 2,1 kg",5073,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3080,"4299,00€",0%,i7,7,Intel,0,32,2.1,Boulanger (FR)
51,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157080,"17,3"" Full HD (43,9 cm) - 300 Hz",300 Hz,"Intel Core i7-10870H-2,2 GHz / Turboboost jusq...",SSD 1 To,"Mémoire vive 32 Go - 2,9 kg",4483,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3080 GDDR6 16 Go,"3799,00€",0%,i7,7,Intel,0,32,2.9,Boulanger (FR)
49,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAsus\n\...,https://www.boulanger.com/ref/1156755,"17,3"" Full HD (43,9 cm) - 300 Hz",300 Hz,AMD Ryzen 7 5800H : 3.0 GHz Turbo 4.3 GHz 16Mo...,SSD 1 To (en PCi-Express),"Mémoire vive 32 Go - 3,0 kg",3893,chez vous dès vendredi 12 mars,Le + : Nvidia GeForce RTX 3080 GDDR6 16GB,"3299,00€",10%,Ryzen 7,7,AMD,10,32,3.0,Boulanger (FR)
70,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAsus\n\...,https://www.boulanger.com/ref/1156752,"17,3"" Full HD (43,9 cm) - 300 Hz",300 Hz,AMD Ryzen 9 5900HX : 3.3 GHz Turbo 4.6 GHz 16M...,SSD 1 To (en PCi-Express),"Mémoire vive 16 Go - 3,0 kg",2713,indisponible,Le + : NVIDIA GeForce RTX 3070 8 Go,"2299,00€",0%,Ryzen 9,9,AMD,0,16,3.0,Boulanger (FR)
72,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAsus\n\...,https://www.boulanger.com/ref/1156743,"15,6"" Full HD (39,6 cm) - 144 Hz",144 Hz,AMD Ryzen 9 5900HX : 3.3 GHz Turbo 4.6 GHz 16M...,SSD 1 To (en PCi-Express),"Mémoire vive 16 Go - 2,5 kg",2359,indisponible,Le + : NVIDIA GeForce RTX 3070 8 Go,"1999,00€",0%,Ryzen 9,9,AMD,0,16,2.5,Boulanger (FR)


In [32]:
df_eg['store']= 'Jumia (EG)'
exchange_factor_eg= 0.06
df_eg.price= df_eg.price.map(lambda x: round(x*exchange_factor_eg))
df_eg.head(1)

,url,text,discount,original_price_EPG,brand,brand_url,price,discount_value,ram_value,cpu_name,cpu_gen,cpu_brand,store
0,https://www.jumia.com.eg/g3-15-3500-gaming-lap...,DELL G3 15-3500 Gaming Laptop - Intel Core I5 ...,18%,"EGP 15,495",DELL,https://www.jumia.com.eg/dell/,930,18,8,I5,5,Intel,Jumia (EG)


In [33]:
columns_to_merge= ['text',	'url', 'price', 'cpu_name',	'cpu_gen', 'cpu_brand', 'discount_value', 'store']
a= df_eg[columns_to_merge]
b= df_fr[columns_to_merge]
c= df_sa[columns_to_merge]

In [34]:
df_merged= a.append(b).append(c)
df_merged.head()

,text,url,price,cpu_name,cpu_gen,cpu_brand,discount_value,store
0,DELL G3 15-3500 Gaming Laptop - Intel Core I5 ...,https://www.jumia.com.eg/g3-15-3500-gaming-lap...,930,I5,5,Intel,18,Jumia (EG)
1,Acer Nitro 5 Laptop - Intel Core I5 10TH - 8GB...,https://www.jumia.com.eg/nitro-5-laptop-intel-...,870,I5,5,Intel,19,Jumia (EG)
2,Lenovo Legion 5-15IMH05H Gaming Laptop - Intel...,https://www.jumia.com.eg/legion-5-15imh05h-gam...,1193,I7,7,Intel,20,Jumia (EG)
3,DELL G5 15-5500 Gaming Laptop - Intel Core I7 ...,https://www.jumia.com.eg/dell-g5-15-5500-gamin...,1014,I7,7,Intel,16,Jumia (EG)
4,Lenovo Legion 5-15IMH05 Gaming Laptop - Intel ...,https://www.jumia.com.eg/legion-5-15imh05-gami...,1011,I7,7,Intel,11,Jumia (EG)


Clean Merged Dataset
1. naming inconsicteny in cpu_name with some refered to as i7 while others as I7
2. there is 1 intel i-1 which is very old and 3 Ryzen 3


In [35]:
#df_merged.cpu_name= df_merged.cpu_name.map(lambda x: x.lower())
df_merged.cpu_gen= df_merged.cpu_gen.astype(float)
df_merged.cpu_gen= df_merged.cpu_gen.astype(int)
df_merged= df_merged[df_merged.cpu_gen> 3] # remove super old cpu
df_merged.cpu_gen.value_counts()


7    149
5     34
9     16
Name: cpu_gen, dtype: int64

In [36]:
df_merged.to_csv('./clean/laptops.csv')
df_eg.to_csv('./clean/jumia_store.csv')
df_fr.to_csv('./clean/boulanger_store.csv')
df_sa.to_csv('./clean/amazon_sa_store.csv')

In [37]:
df_merged.sample(5)

,text,url,price,cpu_name,cpu_gen,cpu_brand,discount_value,store
55,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157077,5309,i9,9,Intel,0,Boulanger (FR)
29,DELL G5 15-5500 Gaming Laptop - Intel Core I7 ...,https://www.jumia.com.eg/g5-15-5500-gaming-lap...,1020,I7,7,Intel,6,Jumia (EG)
92,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1129523,1830,i7,7,Intel,32,Boulanger (FR)
18,Lenovo Ideapad 15IMH05 3 Gaming Laptop - Intel...,https://www.jumia.com.eg/lenovo-ideapad-15imh0...,840,I5,5,Intel,13,Jumia (EG)
46,لاب توب اتش بي بافليون للألعاب 15 بوصة ميكرو إ...,https://www.amazon.sa/i5-9300H%D8%8C-%D8%AC%D9...,845,i5,5,Intel,0,Amazon (SA)


In [38]:
df_fr.sample(10)

,text,url,display,refresh_rate,cpu,storage,ram,price,earliest_arrival,ports,price_before_discount,discount,cpu_name,cpu_gen,cpu_brand,discount_value,ram_value,weight_KG,store
45,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tLenovo\...,https://www.boulanger.com/ref/1147838,"15,6"" Full HD (39,6 cm) - 144 Hz",144 Hz,"Intel Core i7 - 10875H : 2,3 GHz / Turboboost ...",SSD 1 To + SSD de 512 Go,"Mémoire vive 16 Go - 2,3 kg",3539,chez vous dès vendredi 12 mars,NaN,"2999,00€",0%,i7,7,Intel,0,16,2.3,Boulanger (FR)
44,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1152048,"17,3"" Full HD (43,9 cm) - 300 Hz",300 Hz,Intel Core i7,SSD 1 To,"Mémoire vive 32 Go - 2,4 kg",3185,chez vous dès vendredi 12 mars,Le + : Écran type IPS 300Hz et carte Graphiqu...,"3999,00€",- 32%,i7,7,Intel,32,32,2.4,Boulanger (FR)
34,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAsus\n\...,https://www.boulanger.com/ref/1156744,"15,6"" Full HD (39,6 cm) - 240 Hz",240 Hz,"Intel Core i7-11370H : 3,3 GHz Turbo 4,8 GHz 1...",SSD 1 To (en PCi-Express),"Mémoire vive 16 Go - 2,0 kg",2713,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3070 8 Go,"2299,00€",0%,i7,7,Intel,0,16,2.0,Boulanger (FR)
94,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1128022,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,"Intel Core i7 - 9750H : 2,6 GHz / Turboboost j...",Disque dur 1 To et SSD 512 Go (en PCI-Express),"Mémoire vive 16 Go - 2,6 kg",1888,sous 6 jours ouvrés,Le + : NVIDIA GeForce RTX2060 avec 6 Go,"2249,00€",- 28%,i7,7,Intel,28,16,2.6,Boulanger (FR)
2,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157099,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,SSD 1 To,"Mémoire vive 32 Go - 2,3 kg",2241,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3060 GDDR6 6 Go,"1899,00€",0%,i7,7,Intel,0,32,2.3,Boulanger (FR)
52,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tLenovo\...,https://www.boulanger.com/ref/1147839,"15,6"" Full HD (39,6 cm) - 144 Hz",144 Hz,"Intel Core i7 - 10875H : 2,3 GHz / Turboboost ...",SSD 2 To,"Mémoire vive 32 Go - 2,3 kg",4365,chez vous dès vendredi 12 mars,NaN,"3699,00€",0%,i7,7,Intel,0,32,2.3,Boulanger (FR)
91,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAsus\n\...,https://www.boulanger.com/ref/1116475,"15"" (38,1 cm) - 144 Hz",144 Hz,"Intel Core i7-8750H : 2,2 GHz ; Turbo 4,1 GHz ...",Disque dur 1 To et SSD 256 Go (en PCI-Express),"Mémoire vive 16 Go - 2,4 kg",1805,sous 6 jours ouvrés,Le + : NVIDIA GeForce RTX2070 avec 8 Go,"2599,50€",- 41%,i7,7,Intel,41,16,2.4,Boulanger (FR)
64,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tAcer\n\...,https://www.boulanger.com/ref/1151964,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,Intel Core i7 - 10750H,SSD 512 Go,"Mémoire vive 16 Go - 2,9 kg",1769,chez vous dès vendredi 12 mars,Le + : Un mois d'abonnement Xbox Game Pass in...,"1799,00€",- 16%,i7,7,Intel,16,16,2.9,Boulanger (FR)
92,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1129523,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,"Intel Core i7 - 9750H : 2,6 GHz / Turboboost j...",Disque dur 1 To et SSD 256 Go (en PCI-Express),"Mémoire vive 16 Go - 2,6 kg",1830,sous 6 jours ouvrés,Le + : NVIDIA GeForce RTX2070 avec 8 Go,"2299,00€",- 32%,i7,7,Intel,32,16,2.6,Boulanger (FR)
68,PC Gamer \n\t\t\t\t\t\t\n\t\t\t\t\t\t\tMSI\n\t...,https://www.boulanger.com/ref/1157097,"17,3"" Full HD (43,9 cm) - 144 Hz",144 Hz,"Intel Core i7-10870H-2,2 GHz / Turboboost jusq...",SSD 1 To,"Mémoire vive 32 Go - 2,9 kg",2595,chez vous dès vendredi 12 mars,Le + : NVIDIA GeForce RTX 3070 GDDR6 8 Go,"2399,00€",- 8%,i7,7,Intel,8,32,2.9,Boulanger (FR)
